# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Week 6: Generating Texts with Transformers</font>

# <font color="#003660">Notebook 1: Writing with Transformers</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>
<center>
<div>
    <font color="#085986"><b>By the end of this lesson, you ...</b><br><br>
        ... you know how to generate texts with a decoder model, <br>
        ... understand how we can select tokens using search (greedy or beam) or sampling.
    </font>
</div>
</center>
</p>

The following content is heavily inspired by the following excellent sources:


*   Tunstall et al. (2021): Natural Language Processing with Transformers. O'Reilly. https://www.oreilly.com/library/view/natural-language-processing/9781098103231/
*   Hugging Face (2021): Transformer Models - Hugging Face Course. https://huggingface.co/course/



# How does text generation work?

One of the most fascinating things about Transformers is their ability to generate natural language text. A famous example is OpenAI’s GPT-2, which when given the prompt (Tunstall et al. (2021))

> *In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.*

was able to generate a compelling news article about talking unicorns:

> *The scientist named the population, after their distinctive horn, Ovid’s Unicorn. These four-horned, silver-white unicorns were previously unknown to science. Now, after almost two centuries, the mystery of what sparked this odd phenomenon is finally solved. Dr. Jorge Pérez, an evolutionary biologist from the University of La Paz, and several companions, were exploring the Andes Mountains when they found a small valley, with no other animals or humans. Pérez noticed that the valley had what appeared to be a natural fountain, surrounded by two peaks of rock and silver snow. Pérez and the others then ventured further into the valley. “By the time we reached the top of one peak, the water looked blue, with some crystals on top,” said Pérez. Pérez and his friends were astonished to see the unicorn herd. These creatures could be seen from the air without having to move too much to see them - they were so close they could touch their horns. While examining these bizarre creatures the scientists discovered that the creatures also spoke some fairly regular English*



It is remarkable that this text was generated by a model trained without explicit supervision. Through a self-supervised way of learning to predict next words in millions of documents from the Internet, languages models like GPT-2 are able to generate new texts, which are almost indistinguishable from human-written texts.

For NLP tasks like sequence classification, generating predictions is fairly straightforward; the model encodes an input text in a vector and uses this vectors as features for a standard classification task, i.e., it outputs a single or multiple logits, probabilities, or class labels.

The output of a text generation model, in contrast, is not a vector of probabilities or class labels, but again a text sequence. Producing a coherent text sequence requires an extra decoding step (probabilities to text) and poses a number of challenges. For example, instead of simply passing inputs once through the forward pass of a model, the prediction and decoding must be done iteratively and in an autoregressive way. 

Language models are typically pretrained to estimate the probability of a sequence of tokens `y1, y2, ... yt` occurring in the text, given some initial text prompt `x`:

<center><img width=350 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/lm_conditional_prob.png"/><br></center>

We can easily extend the next-token prediction task to generate text sequences of arbitrary length by iteratively repeating the process, taking the output of step `t` as the input of step `t+1`. The Figure below illustrates this process. We start with a prompt like “Transformers are the” and use the model to predict the next token. Once we have determined the next token, we append it to the prompt and then use the new input sequence to generate another token. We do this until we have reached a special end of sequence (EOS) token - for example a full stop (.) - or a predefined maximum length.

<center><img width=600 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/transformers_are_the.png"/><br></center>

For the above process we need a decoding method that determines which token is selected at each timestep. Since the language model head produces a logit $z_{ti}$ per token in the vocabulary at each step, we can get the probability distribution over the next possible token  by taking the softmax of this distribution:

<center><img width=300 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/softmax_zti.png"/><br></center>

Yet, the goal of most decoding methods is to find the most likely *overall* sequence. Since there does not exist an algorithm that can find the optimal decoded sequence in polynomial time, we have to rely on heuristics such as **greedy search** or **beam search** to find likely sequences.


Let's try this out with Hugging Face 🤗. We will use a **GPT-2** decoder model for this.

<center><img width=600 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/decoder.png"/><br></center>

# Import Packages

As always, we first need to load a number of required Python packages:
- `pandas` provides high-performance, easy-to-use data structures and data analysis tools.
- `numpy` is a library adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
- `transformers` provides general-purpose architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with over 32+ pretrained models in 100+ languages.
- `torch` is an open source machine learning library used for applications such as computer vision and natural language processing, primarily developed by Facebook's AI Research lab. 

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load Pre-trained Model

First, we load a model for causal language modeling and a corresponding tokenizer from the model hub.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
model_name = "gpt2-xl"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Greedy Search Decoding

The simplest decoding method to yield sequences of tokens from a model’s raw predictions is to greedily select the token with the highest probability at each timestep:

<center><img width=300 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/greedy_search.png"/><br></center>

The following diagram illustrates greedy search decoding with a simple example (Source: https://huggingface.co/blog/how-to-generate). Note that in reality we consider all known words at each step (and not only 3). Here, greedy search would decode the sequence "The nice woman".

<center><img width=500 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/greedy_ex.png"/><br></center>

To generate new text, a language model needs a user-specified input prompt.

In [ ]:
input_txt = "Transformers are the"

In [ ]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)

In [ ]:
input_ids

Although the Hugging Face library provides a `generate()` function for autoregressive models like GPT-2, let’s implement the decoding step using greedy search by hand to understand what happens in the backstage.

First, let's process the tokenized input with a simple forward pass through the model.

In [ ]:
output = model(input_ids=input_ids)

The `output` contains an element named `logits`, which is a vector of raw (non-normalized) predictions generated by the model. In the code below, we extract the logits after the last token (index -1) of the first input batch (index 0) and normalize it to probabilities using the `softmax()` function.

In [ ]:
next_token_probs = torch.softmax(output.logits[0, -1, :], dim=-1)
next_token_probs

This vector of probabilities has as many entries as we have words in the model's vocabulary.

In [ ]:
len(next_token_probs)

Let's sort the probabilities and display the five most likely next words after the prompt...

In [ ]:
next_token_probs_sorted = torch.argsort(next_token_probs, dim=-1, descending=True)
next_token_probs_sorted[0:5]

In [ ]:
tokenizer.decode(next_token_probs_sorted[0:5])

The five words above are the most likely options for the first token of the generated text. Now let's repeat the process in an autoregressive way to generate a whole sequence of tokens.

In [ ]:
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
      
        # Feed input prompt (plus already generated tokens) into model 
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)

        # Select logits of the last token of the first batch and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        
        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        
        # Append most likely next token to input, and repeat
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

pd.DataFrame.from_records(iterations)

The above loop is useful for understanding what happens under the hood. In real uses cases, however, we would rather use Hugging Face's `generate()` function.

In [ ]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_length=11, do_sample=False)
print(tokenizer.decode(output[0]))

Let's do the same with the 🦄 prompt from above...

In [ ]:
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)

output = model.generate(input_ids, max_length=256, do_sample=False)

print(tokenizer.decode(output[0]))

One common issue with greedy search decoding is that the output tends to comprise repetitive sequences. For instance, in the above example the fragment "The researchers were surprised to find that the unicorns were able" appears multiple times.

# Beam Search Decoding

Instead of selecting the individual token with the highest probability at each step, beam search keeps track of the top-`b` most probable next tokens at each step. The next set of beams are chosen by considering all possible next token extensions of the existing set and selecting the `b` most likely extensions. The process is repeated until we reach the maximum length or an EOS token, and the most likely overall sequence is selected by ranking the beams according to their log-probabilities.

Using the example from above, the following diagram illustrates beam search decoding with b=2 and a maximum sequence length of 3 (Source: https://huggingface.co/blog/how-to-generate). Again, note that in reality we would consider all known words at each step (and not only 3). In contrast to greedy search, beam search would decode the sequence "The dog has".

<center><img width=500 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/beam_ex.png"/><br></center>

Let’s calculate and compare the log-probabilities (for mathematical reasons, we work with log-probabilities instead of probbailities here) of the text generated by greedy and beam search to see if beam search can improve the overall sequence probability. 

We first need two helper functions to calculate the overall sequence probability. The function below calculates the log-probability for a single token.

In [ ]:
# Calculate the log-probability of a single token (from Chapter 8 of Tunstall et al. (2021))
def token_logprob(logits, predictions):
    logp = F.log_softmax(logits, dim=-1)
    logp_prediction = torch.gather(logp, 2, predictions.unsqueeze(2)).squeeze(-1)
    return logp_prediction

And the following function calculates the total log-probability of a sequence by just summing up the log-probabilities for each token.

In [ ]:
# Calculate the log-probability of a whole sequence (from Chapter 8 of Tunstall et al. (2021))
def sequence_logprob(model, predictions, input_len=0):
    with torch.no_grad():
        output = model(predictions)
        log_probs = token_logprob(output.logits[:, :-1, :], predictions[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

Let’s use these functions to first calculate the log-probability of the Greedy search decoder for the 🦄 prompt.

In [ ]:
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)

In [ ]:
output_greedy = model.generate(input_ids, max_length=128, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

In [ ]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
logp

Now let's repeat the same with Beam search...

In [ ]:
output_beam = model.generate(input_ids, max_length=256, num_beams=5,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

In [ ]:
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
logp

We can see that we get a better overall log-probability (higher is better) with Beam search than we did with Greedy search. 

However we can see that Beam search also suffers from repetitive text ("a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains"). One way to address this is to impose an n-gram penalty with the `no_repeat_ngram_size` parameter that tracks which n-grams have been seen and sets the next-token probability to zero if it would produce a previously seen n-gram.

In [ ]:
output_beam = model.generate(input_ids, max_length=256, num_beams=5,
                             do_sample=False, no_repeat_ngram_size=2)
print(tokenizer.decode(output_beam[0]))

In [ ]:
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
logp

Although the log-probability went down, the text reads much better!

# Decoding with Sampling

One solution to reduce repetitions while improving diversity is to use sampling instead of deterministic greedy/beam search.

<center><br><br><img width=600 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/urn_lm.png"/><br></center>

## Random Sampling

The simplest sampling method is to **randomly sample** from the model output’s probability distribution over the full vocabulary at each timestep (`|V|` is the size of the vocabulary):

<center><img width=300 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/random_sampling.png"/><br></center>

We can extend this to control the diversity of the output by adding a **temperature** parameter `T` that rescales the logits before taking the softmax:

<center><img width=300 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/random_sampling_w_temp.png"/><br></center>

With temperature we can control the shape of the probability distribution. Low temperature means that the tokens with high probability get boosted, while the probabilities of less likely tokens get damped. When we increase the temperature the distribution smooths out and the probabilities get closer to each other. This effect is illustrated in the figure below.

<center><img width=600 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/temp.png"/><br></center>

Let's try out random sampling with a relatively high temperature of T=2.

In [ ]:
torch.manual_seed(42)
output_temp = model.generate(input_ids, max_length=128, do_sample=True,
                             temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

With a high temperature the model seems to produce mostly gibberish. 

Let’s see what happens, when we cool down the temperature...

In [ ]:
torch.manual_seed(42)
output_temp = model.generate(input_ids, max_length=128, do_sample=True,
                             temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

This is significantly more coherent and even includes a quote! 

The main lesson we can draw from temperature is that it allows us to control the quality of the samples, but there’s always a trade-off between coherence (low temperature) and diversity (high temperature).

By the way: Try to turn off the manual random seed in the code above. As you will see, text generation using sampling is not deterministic.

## Top-k and Nucleus (Top-p) Sampling

Top-k and nucleus (top-p) sampling are two popular alternatives or extensions to using random sampling. In both cases the basic idea is to restrict the number of possible tokens we can sample from at each timestep.

To see how this works, let’s first visualize a cumulative probability distribution of the model’s outputs.

<center><img width=800 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/topk_nucleus_sampling.png"/><br></center>

Looking at the left diagram, we can see that the chance of picking the token with the highest probability (the isolated bar at 10^-1) is 10%. 

The right diagram shows the cumulative sum of the probabilities of the 10,000 most likely tokens (sorted by descending probabilty). For example, there is a 99% chance of picking any of the 2,000 tokens with the highest probability. 

Or in other words: There is only a 1% chance of not picking any of the tokens that are not in the top-2,000. Although this number might appear small, they become important because we sample often when generating long texts. So even if there is only a 1% or 0.1% chance, if we sample hundreds of times there is a significant chance of picking an unlikely token at some point. And picking such tokens can badly influence the quality of the generated text. 

The idea of **top-k sampling** is to avoid the low probability choices by only sampling from the `k` tokens with the highest probability at each step. This puts a fixed cutoff on the long tail of the distribution. 

In [ ]:
torch.manual_seed(42)
output_topk = model.generate(input_ids, max_length=256, do_sample=True,
                             top_k=50)
print(tokenizer.decode(output_topk[0]))

Instead of defining a fixed cutoff, we can use a dynamic one with **nucleus or top-p sampling**. Here, we set a *condition* when to cutoff. This condition is when a certain probability mass in the selection is reached. For example, when we set the cutoff condition to 95%, we order all tokens by probability and add one token after another from the top list until the sum of probabilities of the selected tokens is 95%.

In [ ]:
torch.manual_seed(42)
output_topp = model.generate(input_ids, max_length=128, do_sample=True,
                             top_p=0.90)
print(tokenizer.decode(output_topp[0]))

You can also combine the two approaches. Setting `top_k=50` and `top_p=0.9` corresponds to the rule of choosing tokens with a probability mass that is 90% but at most 50 tokens.